## Les bryggeridata

In [4]:
import json
import shapely
from shapely.geometry import asShape, shape
import geopandas as gpd


In [ ]:
#funksjoner for å fikse opp dataene

#oversett alt annet enn punkt til punkt
def to_point(row):
    if row['geometry'].geom_type == 'Point':
        return row['geometry']
    return row['geometry'].centroid

#set operator-taggen
def fix_operator(row):
    if row['operator'] is None:
        return row['name']
    return row['operator']

In [19]:
# les inn bryggeriene fra en geojson-fil
df = gpd.read_file('./data/osm_breweries_norway_08032018.geojson')

#fiks opp kolonner
df['geometry'] = df.apply(to_point, axis=1)
df['operator'] = df.apply(fix_operator, axis=1)

# velg ut de kolonnene vi faktisk e
df = df[['name', 'operator', 'geometry']]



print(df)


#print(df)

                                              name  \
0                             Lervig Aktiebryggeri   
1                                 Ringnes Bryggeri   
2                                  Borg Bryggerier   
3                                  Haandbryggeriet   
4                             E. C. Dahls Bryggeri   
5                          Christianssand Brygghus   
6                         Christianssands Bryggeri   
7                                    Aass bryggeri   
8                           Little Brother Brewery   
9                                  Sagene Bryggeri   
10                                   Ægir Bryggeri   
11                                To Tårn Bryggeri   
12                                      Finsehytta   
13                           Inderøy Gårdsbryggeri   
14                                    Ego Brygghus   
15                               Hubertus bryggeri   
16                                 Laagen Bryggeri   
17                          

In [2]:
# åpne rett fil fra filsystemet
with open('./data/osm_breweries_norway_08032018.geojson', 'r') as infile:
    featurecollection = json.loads(infile.read().decode('utf-8-sig'))
    
    
    #featurecollection = json.loads(infile.read())
    #featurecollection = json.loads('{"features":[{"geometry": {"type": "line"}}]}')
    #print featurecollection
    for feature in featurecollection['features']:
        if feature['geometry']['type'] != 'Point':
            shape = asShape(feature['geometry'])
            print shape.centroid
    #        print feature

POINT (5.746142767034293 58.94570846949821)
POINT (10.93189032774258 59.99209675567997)
POINT (11.11239512048502 59.29057490401918)
POINT (10.23138505977879 59.74216480250969)
POINT (10.42700137022198 63.44091298857254)
POINT (7.993930033407581 58.14454205074)
POINT (7.979171369885761 58.14782034059565)
POINT (10.21120833326642 59.7414731548904)
POINT (10.74483330963815 59.91937024420984)
POINT (10.7563746337984 59.93101542900136)
POINT (7.107480454516267 60.85856031977224)
POINT (10.3750363 63.42308775)
POINT (7.50688916808432 60.59802349678374)
POINT (11.18630429353732 63.86768807849006)
POINT (11.0096643047558 59.24338027980776)
POINT (9.187306565894565 62.0235404239768)
POINT (10.06253535156395 61.54886516787159)
POINT (10.25044027181518 59.78318054707483)
POINT (5.914326512121155 59.06612504249627)
